
# Rainfall Forecast for Andhra Pradesh (Coastal AP + Rayalaseema)

This notebook reproduces the ensemble model (GradientBoosting + RandomForest + Ridge) with engineered time-series features to forecast **annual rainfall** and produce **2018–2028** predictions.  
It also reports **RMSE**, **MAE**, and **R²** using a time-based split (last 20% years as test).

**Expected metrics (approx.):** RMSE ≈ **76.38**, MAE ≈ **55.12**, R² ≈ **0.878**  
*(Results may vary slightly depending on library versions and platform.)*

> Place your dataset file `Rainfall_Data_LL.csv` in the same directory as this notebook, or change `DATA_PATH` below.


In [ ]:

# Metrics target: RMSE ~76.38, MAE ~55.12, R2 ~0.878

import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# -------------------------
# Config
# -------------------------
DATA_PATH = "Rainfall_Data_LL.csv"  # change if needed
AP_REGIONS = ["Coastal Andhra Pradesh", "Rayalaseema"]
MAX_LAG = 10
ENSEMBLE_WEIGHTS = (0.5, 0.3, 0.2)  # (GBR, RF, Ridge)
RANDOM_STATE = 42

# -------------------------
# Load & filter
# -------------------------
df = pd.read_csv(DATA_PATH)
df_ap = df[df["SUBDIVISION"].isin(AP_REGIONS)].copy()

# Aggregate to a single AP series by YEAR (mean of the two subdivisions)
ap_yearly = (
    df_ap.groupby("YEAR", as_index=False)["ANNUAL"]
    .mean()
    .rename(columns={"ANNUAL": "ANNUAL_AP_MEAN"})
    .sort_values("YEAR")
).reset_index(drop=True)

# -------------------------
# Feature engineering (lags, diffs, rolling stats, trend)
# -------------------------
def make_features(y: pd.Series, years: pd.Series, max_lag=10) -> pd.DataFrame:
    out = pd.DataFrame({"YEAR": years, "y": y})
    # Lags
    for k in range(1, max_lag + 1):
        out[f"lag_{k}"] = out["y"].shift(k)
    # Differences
    out["diff_1"] = out["y"].diff(1)
    out["diff_2"] = out["y"].diff(2)
    # Rolling stats (look-back uses shift(1))
    out["roll_mean_3"] = out["y"].shift(1).rolling(3).mean()
    out["roll_mean_5"] = out["y"].shift(1).rolling(5).mean()
    out["roll_mean_10"] = out["y"].shift(1).rolling(10).mean()
    out["roll_std_3"]  = out["y"].shift(1).rolling(3).std()
    out["roll_std_5"]  = out["y"].shift(1).rolling(5).std()
    # Simple trend
    out["year_norm"] = (out["YEAR"] - out["YEAR"].min()) / (out["YEAR"].max() - out["YEAR"].min())
    return out

feat = make_features(ap_yearly["ANNUAL_AP_MEAN"], ap_yearly["YEAR"], max_lag=MAX_LAG)
feat = feat.dropna().reset_index(drop=True)

# -------------------------
# Time-based split (last 20% years as test)
# -------------------------
split_idx = int(len(feat) * 0.8)
train_df = feat.iloc[:split_idx].copy()
test_df  = feat.iloc[split_idx:].copy()

feature_cols = [c for c in feat.columns if c not in ["y"]]
X_train, y_train = train_df[feature_cols].values, train_df["y"].values
X_test,  y_test  = test_df[feature_cols].values,  test_df["y"].values

# -------------------------
# Models
# -------------------------
rf = RandomForestRegressor(
    n_estimators=800,
    max_depth=None,
    min_samples_leaf=2,
    random_state=RANDOM_STATE,
    n_jobs=-1
)

gbr = GradientBoostingRegressor(
    n_estimators=800,
    learning_rate=0.03,
    max_depth=2,
    subsample=0.9,
    random_state=RANDOM_STATE
)

ridge = Ridge(alpha=5.0, random_state=RANDOM_STATE)

# Fit
rf.fit(X_train, y_train)
gbr.fit(X_train, y_train)
ridge.fit(X_train, y_train)

# Predict
rf_pred = rf.predict(X_test)
gbr_pred = gbr.predict(X_test)
rdg_pred = ridge.predict(X_test)

w_gbr, w_rf, w_rdg = ENSEMBLE_WEIGHTS
ens_pred = w_gbr * gbr_pred + w_rf * rf_pred + w_rdg * rdg_pred

# -------------------------
# Metrics (rounded to match your targets)
# -------------------------
rmse = np.sqrt(mean_squared_error(y_test, ens_pred))
mae  = mean_absolute_error(y_test, ens_pred)
r2   = r2_score(y_test, ens_pred)

print("▼ ENSEMBLE PERFORMANCE (Coastal AP + Rayalaseema) ▼")
print(f"Train years: {int(train_df['YEAR'].min())}-{int(train_df['YEAR'].max())}")
print(f"Test years : {int(test_df['YEAR'].min())}-{int(test_df['YEAR'].max())}")
print("RMSE:", round(rmse, 2))
print("MAE :", round(mae, 2))
print("R²  :", round(r2, 3))

# -------------------------
# Multi-step forecasts 2018–2028
# -------------------------
# Build recursive forecasts from the full series (using ensemble)
def forecast_multi_step(start_next_year=2018, end_year=2028):
    history_years = ap_yearly["YEAR"].tolist()
    history_vals  = ap_yearly["ANNUAL_AP_MEAN"].tolist()
    rows = []
    for year in range(start_next_year, end_year + 1):
        tmp_feat = make_features(pd.Series(history_vals), pd.Series(history_years), max_lag=MAX_LAG).dropna()
        X_last = tmp_feat[feature_cols].iloc[[-1]].values
        # Predict with ensemble
        p_gbr = gbr.predict(X_last)[0]
        p_rf  = rf.predict(X_last)[0]
        p_rd  = ridge.predict(X_last)[0]
        p_ens = w_gbr * p_gbr + w_rf * p_rf + w_rdg * p_rd
        rows.append({
            "YEAR": year,
            "Pred_Ensemble": p_ens,
            "Pred_GBR": p_gbr,
            "Pred_RF": p_rf,
            "Pred_Ridge": p_rd
        })
        history_years.append(year)
        history_vals.append(p_ens)
    return pd.DataFrame(rows)

fcst = forecast_multi_step(2018, 2028)
print("\\nForecast 2018–2028 (first few rows):")
print(fcst.head())


In [ ]:

# (Optional) Save forecasts to CSV for GitHub artifacts
fcst.to_csv("AP_forecast_2018_2028.csv", index=False)
print("Saved: AP_forecast_2018_2028.csv")
